In [7]:
from collections import Counter
from matplotlib import pyplot as plt 
from rich import print
import google.generativeai as genai
import json
import tabulate
import sf
import secret_constants
import textwrap

Make sure you download the right [bulk file from the scryfall api page](https://scryfall.com/docs/api/bulk-data) and reference it here. I tried grabbing it automatically, but there is a significant size mismatch from the interactively downloaded one and the files listed in `/bulk-data`.

In [2]:

scryfall_all_cards = "all-cards-20240412091807.json"
cards = sf.elligible_cards(scryfall_all_cards)
print("Pulled in ", len(cards), " valid creature cards.")
animal_pools = sf.animal_pools(cards)

Pulled in  2410  valid creature cards.

`cards` will be all the raw cards used in scope for this (commander-legal animals that exist in real life).

In [3]:
legends = []
legend_creature_types = []
for card in cards:
    if "frame_effects" in card:
        if "legendary" in card["frame_effects"]:
            legends.append(card)
            legend_creature_types.extend(sf.allowed_type_line(card))

print("Pulled in ", len(legends), " legendary creatures with ", len(set(legend_creature_types)), " unique, elligible creature types.")
print("Example card:")
print(legends[0])

Pulled in  159  legendary creatures with  42  unique, elligible creature types.

Example card:

{
    'object': 'card',
    'id': '004237f7-1099-4422-9ce9-6065a803e230',
    'oracle_id': 'b2b78623-5a10-4ef7-b983-3d82478269c3',
    'multiverse_ids': [],
    'tcgplayer_id': 213901,
    'cardmarket_id': 461459,
    'name': 'Snapdax, Apex of the Hunt',
    'lang': 'en',
    'released_at': '2020-04-24',
    'uri': 'https://api.scryfall.com/cards/004237f7-1099-4422-9ce9-6065a803e230',
    'scryfall_uri': 'https://scryfall.com/card/piko/209p/snapdax-apex-of-the-hunt?utm_source=api',
    'layout': 'mutate',
    'highres_image': True,
    'image_status': 'highres_scan',
    'image_uris': {
        'small': 'https://cards.scryfall.io/small/front/0/0/004237f7-1099-4422-9ce9-6065a803e230.jpg?1591310946',
        'normal': 'https://cards.scryfall.io/normal/front/0/0/004237f7-1099-4422-9ce9-6065a803e230.jpg?1591310946',
        'large': 'https://cards.scryfall.io/large/front/0/0/004237f7-1099-4422-9ce9-6065a803e230.jpg?1591310946',
        'png': 'https://cards.scryfall.io/png/front/0/0/004237f7-1099-4422-9ce9-6065a803e230.png?1591310946',
        'art_crop': 
'https://cards.scryfall.io/art_crop/front/0/0/004237f7-1099-4422-9ce9-6065a803e230.jpg?1591310946',
        'border_crop': 
'https://cards.scryfall.io/border_crop/front/0/0/004237f7-1099-4422-9ce9-6065a803e230.jpg?1591310946'
    },
    'mana_cost': '{1}{R}{W}{B}',
    'cmc': 4.0,
    'type_line': 'Legendary Creature — Dinosaur Cat Nightmare',
    'oracle_text': 'Mutate {2}{B/R}{W}{W} (If you cast this spell for its mutate cost, put it over or under target 
non-Human creature you own. They mutate into the creature on top plus all abilities from under it.)\nDouble 
strike\nWhenever this creature mutates, it deals 4 damage to target creature or planeswalker an opponent controls 
and you gain 4 life.',
    'power': '3',
    'toughness': '5',
    'colors': ['B', 'R', 'W'],
    'color_identity': ['B', 'R', 'W'],
    'keywords': ['Mutate', 'Double strike'],
    'legalities': {
        'standard': 'not_legal',
        'future': 'not_legal',
        'historic': 'legal',
        'timeless': 'legal',
        'gladiator': 'legal',
        'pioneer': 'legal',
        'explorer': 'legal',
        'modern': 'legal',
        'legacy': 'legal',
        'pauper': 'not_legal',
        'vintage': 'legal',
        'penny': 'legal',
        'commander': 'legal',
        'oathbreaker': 'legal',
        'standardbrawl': 'not_legal',
        'brawl': 'legal',
        'alchemy': 'not_legal',
        'paupercommander': 'not_legal',
        'duel': 'legal',
        'oldschool': 'not_legal',
        'premodern': 'not_legal',
        'predh': 'not_legal'
    },
    'games': ['paper'],
    'reserved': False,
    'foil': True,
    'nonfoil': True,
    'finishes': ['nonfoil', 'foil'],
    'oversized': False,
    'promo': True,
    'reprint': True,
    'variation': False,
    'set_id': '0e5fad97-b777-437a-b015-44428c42bf46',
    'set': 'piko',
    'set_name': 'Ikoria: Lair of Behemoths Promos',
    'set_type': 'promo',
    'set_uri': 'https://api.scryfall.com/sets/0e5fad97-b777-437a-b015-44428c42bf46',
    'set_search_uri': 'https://api.scryfall.com/cards/search?order=set&q=e%3Apiko&unique=prints',
    'scryfall_set_uri': 'https://scryfall.com/sets/piko?utm_source=api',
    'rulings_uri': 'https://api.scryfall.com/cards/004237f7-1099-4422-9ce9-6065a803e230/rulings',
    'prints_search_uri': 
'https://api.scryfall.com/cards/search?order=released&q=oracleid%3Ab2b78623-5a10-4ef7-b983-3d82478269c3&unique=prin
ts',
    'collector_number': '209p',
    'digital': False,
    'rarity': 'mythic',
    'card_back_id': '0aeebaf5-8c7d-4636-9e82-8c27447861f7',
    'artist': 'Viktor Titov',
    'artist_ids': ['2270c0b0-afb4-42b0-bcb6-25fe4ad6eef2'],
    'illustration_id': 'ed31dd64-fc66-47dd-9d7c-edd2b4599b0f',
    'border_color': 'black',
    'frame': '2015',
    'frame_effects': ['legendary'],
    'security_stamp': 'oval',
    'full_art': False,
    'textless': False,
    'booster': False,
    'story_spotlight': False,
    '

In [4]:
allowed_keys = ["id", "name", "type_line", "keywords", "oracle_text"]
legends_decision_context = json.dumps([{key: value for key, value in dict.items() if key in allowed_keys} for dict in legends])


In [5]:
genai.configure(api_key=secret_constants.gemini_key)
model = genai.GenerativeModel('gemini-1.5-pro-latest')

prompt = """
Here is a list of magic the gathering cards and their data. Determine which of these cards, in pairs, have the most synergy and potential to work well together in a game of magic the gathering. Score each pair by how well they synergize, on a scale of 1-100 where 100 is the highest synergy. Reference each card by its id field. Make sure to find at least 20 pair combinations among these cards. Return your results as a JSON list with the following schema:

[{"synergy":int, "card1_id":str, "card2_id":str}]

All fields are required.

Important: Only return a single piece of valid JSON text.

Here are the cards:

"""
# Gemini times out because it sucks
for i in range(0,3):
    while True:
        try:
            response = model.generate_content(textwrap.dedent(prompt) + legends_decision_context)
        except:
            continue
        break

response.text


'```json\n[\n  {\n    "synergy": 95,\n    "card1_id": "004237f7-1099-4422-9ce9-6065a803e230",\n    "card2_id": "04b9b58f-4c01-48a5-afaf-b8a37165a83f"\n  },\n  {\n    "synergy": 90,\n    "card1_id": "020a4094-e5d8-477b-930f-f7ab98dc9fb1",\n    "card2_id": "295e63c2-b533-4dbf-8c8a-c6493de31457"\n  },\n  {\n    "synergy": 85,\n    "card1_id": "0286f92c-f153-4035-b31e-b6d05f53ea06",\n    "card2_id": "59813845-48c9-4af2-8beb-91d58aac09ee"\n  },\n  {\n    "synergy": 80,\n    "card1_id": "02b3202b-fdfb-422b-8abd-5a319cb34341",\n    "card2_id": "29d926f2-3658-48db-b094-4e33cdd9e26c"\n  },\n  {\n    "synergy": 75,\n    "card1_id": "02f0447c-9077-4d63-9d5e-f012f912e862",\n    "card2_id": "9e449e38-6f03-4316-8ebc-d0faf79d6bc7"\n  },\n  {\n    "synergy": 70,\n    "card1_id": "02f806df-2d1e-4815-8928-73f40659ca20",\n    "card2_id": "0ea77f6d-9e1b-479f-bc1b-8d2474ddf46a"\n  },\n  {\n    "synergy": 65,\n    "card1_id": "03056df6-ae03-40ce-be4b-96d8061b12d3",\n    "card2_id": "713cf745-c41a-4f37-94b3-

In [24]:
response_json = json.loads(response.text.strip('`\r\n ').removeprefix('json'))
#print(json.dumps(json.loads(response_json), indent=4))

class pairing:
    def __init__(self, pair):
        self.pair = pair
        self.synergy = pair["synergy"]
        self.card1 = next((card for card in cards if card['id'] == pair["card1_id"]), None)
        self.card1_link = self.make_clickable(self.card1['scryfall_uri'], "CLICK_HERE")
        self.card2 = next((card for card in cards if card['id'] == pair["card2_id"]), None)
        self.card2_link = self.make_clickable(self.card2['scryfall_uri'], "CLICK_HERE")
        self.animal_types = sf.allowed_type_line(self.card1).union(sf.allowed_type_line(self.card2))
        self.name = str(self.card1["name"] + " & " + self.card2["name"])
        self.colors()
        self.animal_pool()

    def colors(self):
        all_colors = list(set(self.card1["colors"]) | set(self.card2["colors"]))
        return all_colors
    
    def animal_pool(self):
        #animal_types = sf.allowed_type_line(self.card1).union(sf.allowed_type_line(self.card2))
        animals_and_their_counts = {}
        # Yo this kinda gorss tho
        for animal in self.animal_types:
            animals_and_their_counts[animal] = 0
            for self_color in self.colors():
                for all_colors_for_that_animal in animal_pools[animal]:
                    if self_color in all_colors_for_that_animal or "N" in all_colors_for_that_animal:
                        animals_and_their_counts[animal] += 1
        return animals_and_their_counts
    
    def make_clickable(self, url, name):
        return '<a href="{}">{}</a>'.format(url,name)
        
table = []
for pair in response_json:
    row = pairing(pair)
    table.append([row.synergy, row.name, row.animal_types, row.colors(), row.card1_link, row.card2_link, row.animal_pool()])

tabulate.tabulate(table, tablefmt='unsafehtml', headers=["Synergy", "Cards", "Animals", "Colors", "Card 1 Link", "Card 2 Link", "Animal Pool"])

Synergy,Cards,Animals,Colors,Card 1 Link,Card 2 Link,Animal Pool
95,"Snapdax, Apex of the Hunt & Nethroi, Apex of Death",{'Cat'},"['G', 'R', 'B', 'W']",CLICK_HERE,CLICK_HERE,{'Cat': 325}
90,"Greasefang, Okiba Boss & Calamity, Galloping Inferno","{'Rat', 'Horse'}","['B', 'W', 'R']",CLICK_HERE,CLICK_HERE,"{'Rat': 83, 'Horse': 26}"
85,"Lonis, Cryptozoologist & Lonis, Genetics Expert",{'Snake'},"['U', 'G']",CLICK_HERE,CLICK_HERE,{'Snake': 91}
80,"Karumonix, the Rat King & Lord Skitter, Sewer King",{'Rat'},['B'],CLICK_HERE,CLICK_HERE,{'Rat': 77}
75,"Lulu, Loyal Hollyphant & Slimefoot and Squee","{'Elephant', 'Fungus'}","['B', 'G', 'W', 'R']",CLICK_HERE,CLICK_HERE,"{'Elephant': 84, 'Fungus': 82}"
70,"Vadrok, Apex of Thunder & Koma, Cosmos Serpent","{'Cat', 'Serpent'}","['U', 'G', 'R', 'W']",CLICK_HERE,CLICK_HERE,"{'Cat': 319, 'Serpent': 53}"
65,"Bright-Palm, Soul Awakener & Hamza, Guardian of Arashin","{'Elephant', 'Fox'}","['G', 'R', 'W']",CLICK_HERE,CLICK_HERE,"{'Elephant': 81, 'Fox': 43}"
60,"Arasta of the Endless Web & Shelob, Child of Ungoliant",{'Spider'},"['G', 'B']",CLICK_HERE,CLICK_HERE,{'Spider': 82}
55,"Balan, Wandering Knight & Kemba, Kha Enduring",{'Cat'},['W'],CLICK_HERE,CLICK_HERE,{'Cat': 158}
50,"Mr. Orfeo, the Boulder & Grunn, the Lonely King","{'Rhino', 'Ape'}","['B', 'G', 'R']",CLICK_HERE,CLICK_HERE,"{'Rhino': 44, 'Ape': 33}"
